In [1]:
from IPython import display

import torch
import torch.nn as nn
from torch.nn import init
import torchvision
import torchvision.transforms as T
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as dset

import numpy as np

import time
from visdom import Visdom

from lib.VisdomWrapper import *

In [12]:
batch_size = 256
num_epochs = 10

loss = nn.CrossEntropyLoss()
vis = VisdomController()

mnist = dset.MNIST('input', train=True, download=True, transform=T.ToTensor())
balanced_train = DataLoader(mnist, batch_size =batch_size)

mnist_test=dset.MNIST('input', train=False, download=True, transform=T.ToTensor())
balanced_test = DataLoader(mnist_test)

Setting up a new session...


In [3]:
def build_classifier():
    return nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=5, stride=1),
        nn.LeakyReLU(0.01),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(32, 64, kernel_size=5, stride=1),
        nn.LeakyReLU(0.01),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Flatten(),
        nn.Linear(4*4*64, 4*4*64, bias=True),
        nn.LeakyReLU(0.01),
        nn.Linear(4*4*64, 10, bias=True)
    )

In [7]:
def get_optimizer(model):
    optimizer = optim.Adam(model.parameters(), lr=1e-5, betas=(.5, .999))
    return optimizer

In [5]:
def train_classifier(classifier, optimizer, data):
    loss_axis = 0
    for epoch in range(num_epochs):
        for n_batch, (x, y) in enumerate(data):
            if len(x) != batch_size:
                continue
            optimizer.zero_grad
            x = x.cuda()
            y = y.cuda()
            scores = classifier(x)
            out = loss(scores, y)
            out.backward()
            optimizer.step()
            if (n_batch % 100 == 0):
                display.clear_output(True)
                print("Epoch {}, {} / {}".format(epoch, n_batch, len(data)))
                print("Loss: ", out)
                vis.PlotLoss("Gen Loss", loss_axis, out.item())
                loss_axis += 1
            

In [14]:
def test(model, device, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()


    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:
net = build_classifier().cuda()
net.train()
train_classifier(net, get_optimizer(net), balanced_train)


Epoch 9, 200 / 235
Loss:  tensor(0.1334, device='cuda:0', grad_fn=<NllLossBackward>)


In [16]:
device = torch.device("cuda")
test(net, device, balanced_test)

NameError: name 'test_loss' is not defined